# Extracting a mask layer from a LayeredH5

In some analysis, you might want to combine multiple layers into one composite layer to understand how much land is included or excluded based on some exclusion criteria

This tutorial will take you through the process of building out this criteria and using the criteria specifications to extract a boolean mask layer.

### Python Implementation

In [1]:
#Importing libraries
from pathlib import Path

from reVX.handlers.layered_h5 import LayeredH5
from reVX.handlers.geotiff import Geotiff
from reV.supply_curve.exclusions import ExclusionMaskFromDict

First let's add path definitions

In [2]:
#Define paths to data and h5 file
DATA = Path("../data")
H5_PATH = DATA.joinpath("example.h5")

Let's take a look at the layers in the h5 file

In [3]:
h5 = LayeredH5(H5_PATH)
h5.layers

['airport_heliport_setbacks',
 'latitude',
 'longitude',
 'nexrad_green_los',
 'setbacks_pipeline_reference',
 'setbacks_structure_115hh_170rd',
 'setbacks_structure_reference']

To extract a mask layer, we need to define a set of criteria for excluding land.

This implemented via `exclusion dictionaries`.

Keywords such as `exclude_values`, `exclude_range`, `include_values`, `include_range` etc can be specified in the exclusion dictionary to exclude/include specific values and/or range of values

More documentation on these keywords can be found [here](https://nrel.github.io/reV/_autosummary/reV.supply_curve.exclusions.LayerMask.html)

Let's create our first exclusion dictionary!

An exclusion dictionary is a dictionary of layer and keyword specification.

In this example, we want to exclude areas that are occupied by our layers. 

*`exclude_range` is not neccessary in this example but used for demonstration purposes

In [5]:
#Construct exclusion dictionary
excl_dict = {
    "nexrad_green_los": {
        "exclude_values": 1
    },
    "airport_heliport_setbacks": {
        "exclude_values": 1
    },
    "setbacks_pipeline_reference": {
        "exclude_values": 1
    },
    "setbacks_structure_115hh_170rd": {
        "exclude_range": [
            0,
            1
        ]
    },
    "setbacks_structure_reference": {
        "exclude_range": [
            0,
            1
        ]
    }
}

Compute the mask

In [7]:
#Compute mask
mask_ = ExclusionMaskFromDict.run(str(H5_PATH), layers_dict=excl_dict)

The run method also has min_area and kernel arguments use to specify minimum contiguous area and contiguous filter method

To write out the mask layer to a file, we use the `Geotiff` handler.

In [8]:
#specify output path
save_path = DATA.joinpath("composite_mask.tif")

#extract profile info from h5
profile = h5.profile

#write mask to file
Geotiff.write(save_path, profile, mask_)

### Command line implementation

The mask extraction can be done from the command line

First we construction a json exclusion dictionary

`excl_dict.json`
```json
{
    "excl_dict" : {
        "nexrad_green_los": {
            "exclude_values": 1
        },
        "airport_heliport_setbacks": {
            "exclude_values": 1
        },
        "setbacks_pipeline_reference": {
            "exclude_values": 1
        },
        "setbacks_structure_115hh_170rd": {
            "exclude_range": [
                0,
                1
            ]
        },
        "setbacks_structure_reference": {
            "exclude_range": [
                0,
                1
            ]
        }
    }
}
```

Then we run `$ reVX exclusions mask  -h5 "../data/example.h5" --excl_dict_fpath excl_dict.json` on the command line.

This command also has `--min_area` and `--kernel options`